In [1]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [2]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio_selection import PortfolioSelection
from financial_common.risk.risk_type import RiskType
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [4]:
market = ADatabase("market")
fred = ADatabase("fred")
orivault = ADatabase("ori")

In [5]:
## Retrieve existing stocks 
market.connect()
index = market.retrieve("index")
market.disconnect()

In [6]:
## Defining Benchmark
market.connect()
benchmark = market.query("prices",{"ticker":"JPM"})
market.disconnect()
benchmark = Benchmark.convert_to_benchmark(benchmark,"adjclose")

In [7]:
rolling_window = 10
market.connect()
prices = []
skips = 1
model_end = 2021
backtest_end = 2026
for ticker in tqdm(index["ticker"][::skips]): 
    try:
        price = market.query("prices", {"ticker": ticker})
        price["market_cap"] = price["adjclose"] * price["volume"]
        if price.index.size > 150:
            price = p.lower_column(price)
            price = p.utc_date(price)
            price = p.additional_date_columns(price)
            price = price[(price["year"]>=model_end) & (price["year"]<backtest_end)]
            price.sort_values("date", inplace=True)
            price = Metric.STANDARD_DEV.calculate(price,timeframe=rolling_window)
            price = Metric.NEXT_CLOSE.calculate(price,timeframe=rolling_window)
            price = Metric.NEXT_HIGH.calculate(price,timeframe=rolling_window)
            price = Metric.NEXT_LOW.calculate(price,timeframe=rolling_window)
            for member in Indicator:
                price = member.calculate(price,timeframe=rolling_window)
            prices.append(price.dropna())
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

  3%|█████▏                                                                                                                                                                                                          | 282/11272 [00:07<03:58, 46.10it/s]

'adjclose'


 15%|███████████████████████████████▍                                                                                                                                                                               | 1713/11272 [00:47<05:03, 31.53it/s]

'adjclose'


 34%|███████████████████████████████████████████████████████████████████████▎                                                                                                                                       | 3881/11272 [01:50<02:46, 44.44it/s]

...

 46%|████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                              | 5241/11272 [02:34<02:26, 41.25it/s]

'adjclose'


 48%|██████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                            | 5374/11272 [02:37<02:52, 34.18it/s]

'adjclose'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11272/11272 [05:50<00:00, 32.18it/s]


In [8]:
simulation = pd.concat(prices)
simulation["coefficient_of_variation"] = 1
simulation.sort_values("date", inplace=True)
simulation.columns

Index(['date', 'adjclose', 'high', 'low', 'volume', 'ticker', 'market_cap',
       'year', 'month', 'quarter', 'week', 'day', 'weekday', 'adjclose_test',
       'high_test', 'low_test', 'volume_test', 'standard_dev', 'next_close',
       'next_high', 'next_low', 'optimal', 'adr', 'sma', 'sma_corr', 'ema',
       'ema_corr', 'bollinger_upper', 'bollinger_lower', 'momentum', 'roc',
       'williams_r', 'obv', 'vwap', 'market_impact', 'atr',
       'coefficient_of_variation'],
      dtype='object')

In [9]:
portfolios = PortfolioSelection.generate_possible_portfolios(
                                                            ranking_metrics=[
                                                                "ema","sma","roc"
                                                            ]
                                                            ,num_of_groups=[1]
                                                            ,group_percentages=[1]
                                                            ,max_prices=[0.1]
                                                            ,min_prices=[0.01]
                                                            ,max_market_caps = [1000000]
                                                            ,min_market_caps = [10000]
                                                            ,stoplosses=[1]
                                                            ,rolling_windows=[rolling_window]
                                                            ,leverages=[1]
                                                            )
filtered_portfolios = []
for portfolio in portfolios:
    if portfolio.timeframe.value == "day" and "blacklist" in portfolio.selection_type.label \
        and portfolio.allocation_type.label == "risk" \
        and portfolio.ranking_metric != "optimal" \
        and portfolio.position_type.label == "long" and ("standard_dev" in portfolio.grouping_type.value) \
        and portfolio.max_price > portfolio.min_price:
        filtered_portfolios.append(portfolio)

In [ ]:
analysises = []
for portfolio in tqdm(filtered_portfolios):
    try:
        trades = portfolio.trades(simulation.copy())
        trades = p.additional_date_columns(trades)
        trades = trades.groupby("weekday").agg({"date":"last","weighted_return":"mean","return":"mean"}).reset_index().sort_values("date")
        trades["date"] = trades.apply(lambda row: datetime.strptime(f"{2025} {10} {row['weekday']+1}", "%Y %W %u"), axis=1)
        trades = p.additional_date_columns(trades)
        trades.sort_values("date")
        performance = portfolio.portfolio(trades.copy(),benchmark.copy())
        metrics = KPI.performance(trades,performance)
        results = {**portfolio.to_dict(),**metrics.copy()}
        results["portfolio"] = portfolio
        analysises.append(results)
    except Exception as e:
        print(str(e))
        continue

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                       | 4/6 [00:51<00:25, 12.90s/it]

In [ ]:
a = pd.DataFrame(analysises)

In [ ]:
metric = "raw_pnl"
dropped_columns = ["date","risk_type","max_price","min_price","max_market_cap","min_market_cap","timeframe","selection_percentage","rolling_window","portfolio"]
filtered_a = a.drop(dropped_columns,axis=1).sort_values(metric,ascending=False)
top = a.sort_values(metric,ascending=False).head(20).to_dict("records")[0]
filtered_a.head(20)

In [ ]:
pm = top["portfolio"]
trades = pm.trades(simulation.copy())
trades = p.additional_date_columns(trades)
trades = trades.groupby("day").agg({"date":"last","weighted_return":"mean","return":"mean","market_cap":"mean"}).reset_index().sort_values("date")
trades = p.additional_date_columns(trades)
portfolio = pm.portfolio(trades.copy(),benchmark.copy())
portfolio = p.additional_date_columns(portfolio)

In [ ]:
averaged_portfolio = portfolio.groupby("weekday").agg({"weighted_return":"mean","return":"mean"}).reset_index()
averaged_portfolio["pv"] = averaged_portfolio["weighted_return"]
plt.plot(averaged_portfolio["pv"])
plt.show()

In [ ]:
orivault.connect()
orivault.drop("results")
orivault.store("results",pd.DataFrame([top]).drop("portfolio",axis=1))
orivault.disconnect()